In [1]:
import tensorflow
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [2]:
def define_discriminator(in_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides = (2,2), padding = 'same', input_shape = in_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))

    opt = Adam(lr = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])

    return model

In [3]:
def define_generator(latent_dim):
    gen_model = Sequential()
    n_nodes = 128 * 7 * 7
    gen_model.add(Dense(n_nodes, input_dim = latent_dim))
    gen_model.add(LeakyReLU(alpha = 0.2))
    gen_model.add(Reshape((7, 7, 128)))
    #upsample to 14x14
    gen_model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding='same'))
    gen_model.add(LeakyReLU(alpha = 0.2))
    #upsample to 28x28
    gen_model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding='same'))
    gen_model.add(LeakyReLU(alpha = 0.2))
    gen_model.add(Conv2D(1, (7, 7), activation = 'sigmoid', padding='same'))

    return gen_model

In [4]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    opt = Adam(lr = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt)

    return model

In [5]:
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X/255.0
    return X

In [6]:
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

In [7]:
def generate_latent_points(latent_dim, n_samples):
    X_input = randn(latent_dim * n_samples)
    X_input = X_input.reshape(n_samples, latent_dim)
    return X_input

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    X_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(X_input)
    y = zeros((n_samples, 1))
    return X, y

In [9]:
def viz_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n, n, 1+i)
        pyplot.axis('off')
        pyplot.imshow(examples[i, :, :, 0], cmap = 'gray_r')
    filename = 'C:/Users/bhatt/OneDrive/Desktop/whisper-main/DS_PP_11Jun/Data_Science_PP_11_June-main/Data_Science_PP_11_June-main/generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

    return pyplot

In [10]:
def summary(g_model, d_model, dataset, latent_dim, epoch, n_samples = 100):
    X_real, y_real = generate_real_samples(dataset, n_samples)

    _, acc_real = d_model.evaluate(X_real, y_real, verbose = 0)

    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)

    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose = 0)

    print('> Accuracy real: %.0f%%, fake: %0.0f%%' % (acc_real*100, acc_fake*100))

    pyplot = viz_plot(X_fake, epoch)

    return pyplot

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs = 100, n_batch = 256):
    bat_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch/2)
    pyplot = []

    for i in range(n_epochs):
        for j in range(bat_per_epoch):
            X_real, y_real = generate_real_samples(dataset, half_batch)

            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

            d_loss_real, _ = d_model.train_on_batch(X_real, y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, y_fake)

            d_loss = d_loss_real + d_loss_fake

            X_gan = generate_latent_points(latent_dim, n_batch)

            y_gan = ones((n_batch, 1))

            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            print('> %d, %d/%d, d = %.3f, g=%.3f' % (i+1, j+1, bat_per_epoch, d_loss, g_loss))

        if (i) % 10 == 0:
            pyplot.append(summary(g_model, d_model, dataset, latent_dim, i))

    return pyplot

In [ ]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

plot = train(g_model, d_model, gan_model, dataset, latent_dim)

4/4 [==============================] - 0s 74ms/step
> 1, 1/234, d = 1.536, g=0.621
4/4 [==============================] - 0s 69ms/step
> 1, 2/234, d = 1.394, g=0.708
4/4 [==============================] - 0s 74ms/step
> 1, 3/234, d = 1.296, g=0.823
4/4 [==============================] - 0s 62ms/step
> 1, 4/234, d = 1.204, g=0.960
4/4 [==============================] - 0s 75ms/step
> 1, 5/234, d = 1.105, g=1.137
4/4 [==============================] - 0s 71ms/step
> 1, 6/234, d = 1.003, g=1.331
4/4 [==============================] - 0s 73ms/step
> 1, 7/234, d = 0.954, g=1.382
4/4 [==============================] - 0s 76ms/step
> 1, 8/234, d = 0.988, g=1.129
4/4 [==============================] - 0s 67ms/step
> 1, 9/234, d = 1.144, g=0.743
4/4 [==============================] - 0s 74ms/step
> 1, 10/234, d = 1.181, g=0.626
4/4 [==============================] - 0s 73ms/step
> 1, 11/234, d = 1.069, g=0.646
4/4 [==============================] - 0s 67ms/step
> 1, 12/234, d = 0.980, g=0.670
4